### This script relies on a active environment with Basemap
If that is not possible, you properly have to outcomment a thing or two.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import geopandas as gpd
from mpl_toolkits.basemap import Basemap
import ezodf

In [2]:
countries = gpd.read_file('../Data Sources/countries (1)/ne_50m_admin_0_countries.shp').fillna(value='None')
globalFreight = pd.read_csv('../Data Sources/Global Air Freight (1)/Air_Transported_Goods.csv',
                            header=4).fillna(value=0)
globalPassengers = pd.read_csv('../Data Sources/Global Passengers Carried (1)/Passengers_Carried.csv',
                               header=4).fillna(value=0)

## Trade flows (Global)

In [3]:
tradeFlows = pd.read_csv('StructuredData.csv',parse_dates=['date'],index_col =0)
# Limiting the data to only 2017 for now.
tradeFlows = tradeFlows[((tradeFlows.date >='2017-01-01') & (tradeFlows.date <= '2017-12-01')) &\
                       (tradeFlows['product'] != 'TOTAL')]
tradeFlows = tradeFlows.reset_index(drop=True)
tradeFlows['product'] = tradeFlows['product'].astype(int)

In [4]:
tradeFlows.head()

,trade_value,product,product_description,date,partner,indicator_type,reporter,reporter_ISO
0,0.770370,1,Animals; live,2017-01-01,World,Export,Albania,ALB
1,0.044056,1,Animals; live,2017-01-01,World,Export,Algeria,DZA
2,0.071136,1,Animals; live,2017-01-01,World,Export,Andorra,AND
3,1242.319413,1,Animals; live,2017-01-01,World,Export,Australia,AUS
4,188.706390,1,Animals; live,2017-01-01,World,Export,Austria,AUT


In [5]:
######################### Restructuring data ###########################
# _______________________________________________________________________

####################### Standard 15 categories #########################

# hsMainCategories = ['animals_&_animal_products','vegetable_products','foodstuffs','mineral_products',
#                     'chemicals_&_allied_industries','Plastics/Rubbers','raw_hides,_skins,_leather_&_furs',
#                    'wood_&_wood_products','textiles','footwear_/_headgear','stone_/_glass','metals',
#                    'machinery_/_electrical','transportation','miscellaneous','all_commodities']

# hsMain = [(1,5),(6,15),(16,24),(25,27),(28,38),(39,40),(41,43),(44,49),
#           (50,63),(64,67),(68,71),(72,83),(84,85),(86,89),(90,97),'TOTAL']

####################### Reduced set of categories #######################

hsMainCategories = ['organic_products','chemicals','material','clothing','metals',
                    'technical','transportation','miscellaneous_goods','all_commodities']

hsMain = [(1,24),(28,38),[(25,27),(39,56),(68,71)],(57,67),(72,83),(84,85),(86,89),(90,97),'TOTAL']

exports = [[] for i in np.arange(0,len(hsMain))]
imports = [[] for i in np.arange(0,len(hsMain))]

for country in tradeFlows.reporter.unique():
    
    # Only considering the relevant observations
    tempTradeFlows = tradeFlows[tradeFlows.reporter==country]
    tempTradeFlows = tempTradeFlows.reset_index(drop=True)
    for i,categories in enumerate(hsMain):
        #print(categories)
        if type(categories) != list: 
            # First export
            if type(categories) != str:

                exports[i].append(round(sum(tempTradeFlows[((tempTradeFlows['product'] >= categories[0]) &\
                                           (tempTradeFlows['product'] <= categories[1])) &\
                                           (tempTradeFlows.indicator_type == 'Export')].trade_value),3))
            else:
                exports[i].append(round(sum(tempTradeFlows[tempTradeFlows.indicator_type=='Export'].trade_value),3))

            # Then import
            if type(categories) != str:

                imports[i].append(round(sum(tempTradeFlows[((tempTradeFlows['product'] >= categories[0]) &\
                                            (tempTradeFlows['product'] <= categories[1])) &\
                                            (tempTradeFlows.indicator_type == 'Import')].trade_value),3))
            else:
                imports[i].append(round(sum(tempTradeFlows[tempTradeFlows.indicator_type=='Import'].trade_value),3))
        
        elif type(categories) == list:
            #Extention
            tempExport = 0
            tempImport = 0
            
            for subCategory in categories:
                if type(subCategory) == str:
                    raise ValueError('You cannot include "Total" in a list of tuples.')
                
                tempExport += round(sum(tempTradeFlows[((tempTradeFlows['product'] >= subCategory[0]) &\
                                           (tempTradeFlows['product'] <= subCategory[1])) &\
                                           (tempTradeFlows.indicator_type == 'Export')].trade_value),3)
                                      
                tempImport += round(sum(tempTradeFlows[((tempTradeFlows['product'] >= subCategory[0]) &\
                                            (tempTradeFlows['product'] <= subCategory[1])) &\
                                            (tempTradeFlows.indicator_type == 'Import')].trade_value),3)
            # Appending at the end
            exports[i].append(tempExport)
            imports[i].append(tempImport)
                
#         else:
#             raise ValueError('The format of the positions are wrong, either provide it as a list of tuples or a single tuple')

combined = exports+imports

In [6]:
tradeFlowData = pd.DataFrame(index = tradeFlows.reporter.unique(),
                             columns=[category+'_export' for category in hsMainCategories]+\
                                     [category+'_import' for category in hsMainCategories])
for i,category in enumerate(combined):
    tradeFlowData.iloc[:,i] = category

In [7]:
tradeFlowData['ISO3'] = tradeFlows.reporter_ISO.unique()

In [8]:
tradeFlowData.head()

,organic_products_export,chemicals_export,material_export,clothing_export,metals_export,technical_export,transportation_export,miscellaneous_goods_export,all_commodities_export,organic_products_import,chemicals_import,material_import,clothing_import,metals_import,technical_import,transportation_import,miscellaneous_goods_import,all_commodities_import,ISO3
Albania,253.592,20.481,504.829,987.659,351.251,106.350,20.772,55.395,2300.329,902.405,484.346,1466.302,571.808,526.633,767.239,366.151,189.804,5274.687,ALB
Algeria,352.286,775.930,33964.004,0.811,11.783,59.750,21.057,5.497,35191.117,9415.691,4257.590,7163.324,939.355,6513.290,11991.787,4394.323,1377.655,46053.016,DZA
Andorra,0.857,1.938,20.368,9.705,5.748,35.502,23.803,19.403,117.324,339.174,180.473,269.969,162.165,38.289,177.221,187.202,122.333,1476.827,AND
Australia,33023.048,11404.581,154373.943,456.601,10231.289,7665.664,3586.913,3858.712,224600.751,15334.305,19711.056,50708.185,10612.758,10622.221,52992.304,40094.228,17040.121,217115.178,AUS
Austria,12558.670,17692.036,28443.476,4700.658,22400.738,45032.580,19517.264,9506.904,159852.324,13515.714,18451.627,36268.453,9584.452,18119.193,37796.299,22006.902,10677.261,166419.902,AUT


In [9]:
tradeFlowData.columns[0:-1]

Index(['organic_products_export', 'chemicals_export', 'material_export',
       'clothing_export', 'metals_export', 'technical_export',
       'transportation_export', 'miscellaneous_goods_export',
       'all_commodities_export', 'organic_products_import', 'chemicals_import',
       'material_import', 'clothing_import', 'metals_import',
       'technical_import', 'transportation_import',
       'miscellaneous_goods_import', 'all_commodities_import'],
      dtype='object')

In [10]:
tradeFlowData.iloc[:,0:-1] = tradeFlowData.iloc[:,0:-1]/1000

In [11]:
tradeFlowData.iloc[:,0:-1] = tradeFlowData.iloc[:,0:-1].round(2)

## Global Freight

In [12]:
globalFreight.shape

(264, 63)

In [13]:
tradeFlowData.shape

(144, 19)

In [14]:
# Let's examine some of the variables which could be expected to have not variation.
print('All Columns: \n\n',globalFreight.columns,'\n')
print('Unique values in "Indicator Name": ',globalFreight['Indicator Name'].unique(),'\n')
print('Unique values in "Indicator Code": ',globalFreight['Indicator Code'].unique(),'\n')

All Columns: 

 Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018'],
      dtype='object') 

Unique values in "Indicator Name":  ['Air transport, freight (million ton-km)'] 

Unique values in "Indicator Code":  ['IS.AIR.GOOD.MT.K1'] 



In [15]:
globalFreight = globalFreight[['Country Name','Country Code','2013','2014','2015','2016','2017']]

In [16]:
globalFreight.head()

,Country Name,Country Code,2013,2014,2015,2016,2017
0,Aruba,ABW,0.00,0.00,0.00,0.00,0.00
1,Afghanistan,AFG,84.62,34.28,33.10,29.01,25.14
2,Angola,AGO,70.43,65.52,46.04,43.94,67.91
3,Albania,ALB,0.00,0.00,0.00,0.00,0.00
4,Andorra,AND,0.00,0.00,0.00,0.00,0.00


In [17]:
print('Countries with a value of zero:\n')
print({year:sum([1 if value == float(0) else 0 for value in globalFreight[year]])\
       for year in ['2013','2014','2015','2016','2017']})
print('\nIn percentage of the number of countries available: \n')
print({year:round(sum([1 if value == float(0) else 0 for value in globalFreight[year]])/globalFreight.shape[0],3)\
       for year in ['2013','2014','2015','2016','2017']})

Countries with a value of zero:

{'2013': 77, '2014': 78, '2015': 81, '2016': 82, '2017': 86}

In percentage of the number of countries available: 

{'2013': 0.292, '2014': 0.295, '2015': 0.307, '2016': 0.311, '2017': 0.326}


## Global Passengers

In [18]:
globalPassengers.shape

(264, 63)

In [19]:
globalPassengers.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,"Air transport, passengers carried",IS.AIR.PSGR,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,"Air transport, passengers carried",IS.AIR.PSGR,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1999127.0,2279341.18,1737962.13,2044188.0,2209428.0,1929907.0,1917924.0,1858558.0,0.0
2,Angola,AGO,"Air transport, passengers carried",IS.AIR.PSGR,0.0,0.0,0.0,0.0,0.0,0.0,...,274869.0,1010194.0,987798.00,1132424.00,1321872.0,1409952.0,1244491.0,1482546.0,1581201.0,0.0
3,Albania,ALB,"Air transport, passengers carried",IS.AIR.PSGR,0.0,0.0,0.0,0.0,0.0,0.0,...,231263.0,768533.0,829778.92,814339.73,865848.0,151632.0,0.0,26634.0,103151.0,0.0
4,Andorra,AND,"Air transport, passengers carried",IS.AIR.PSGR,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Let's examine some of the variables which could be expected to have not variation.
print('All Columns: \n\n',globalPassengers.columns,'\n')
print('Unique values in "Indicator Name": ',globalPassengers['Indicator Name'].unique(),'\n')
print('Unique values in "Indicator Code": ',globalPassengers['Indicator Code'].unique(),'\n')

All Columns: 

 Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018'],
      dtype='object') 

Unique values in "Indicator Name":  ['Air transport, passengers carried'] 

Unique values in "Indicator Code":  ['IS.AIR.PSGR'] 



In [21]:
globalPassengers = globalPassengers[['Country Name','Country Code','2013','2014','2015','2016','2017']]

In [22]:
globalPassengers.head()

,Country Name,Country Code,2013,2014,2015,2016,2017
0,Aruba,ABW,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,2044188.0,2209428.0,1929907.0,1917924.0,1858558.0
2,Angola,AGO,1321872.0,1409952.0,1244491.0,1482546.0,1581201.0
3,Albania,ALB,865848.0,151632.0,0.0,26634.0,103151.0
4,Andorra,AND,0.0,0.0,0.0,0.0,0.0


In [23]:
print('Countries with a value of zero:\n')
print({year:sum([1 if value == float(0) else 0 for value in globalPassengers[year]])\
       for year in ['2013','2014','2015','2016','2017']})
print('\nIn percentage of the number of countries available: \n')
print({year:round(sum([1 if value == float(0) else 0 for value in globalPassengers[year]])/globalPassengers.shape[0],3)\
       for year in ['2013','2014','2015','2016','2017']})

Countries with a value of zero:

{'2013': 62, '2014': 63, '2015': 64, '2016': 63, '2017': 65}

In percentage of the number of countries available: 

{'2013': 0.235, '2014': 0.239, '2015': 0.242, '2016': 0.239, '2017': 0.246}


### Let's compare with the 'countries' shapefile

In [24]:
print('The number of matching entities in the global freight data are:',sum([1 if country in list(countries.NAME_EN) else 0\
                                                                        for country in globalFreight['Country Name']]),'\n')
print('The number of matching entities in the global passenger data are:',sum([1 if country in list(countries.NAME_EN) else 0\
                                                                            for country in globalPassengers['Country Name']]))

The number of matching entities in the global freight data are: 179 

The number of matching entities in the global passenger data are: 179


## Merging the global freight, passenger and trade flow data together

In [25]:
merged1 = tradeFlowData.merge(globalFreight[['Country Code','2017']],left_on = 'ISO3',right_on='Country Code')
merged2 = merged1.merge(globalPassengers[['Country Code','2017']],left_on='ISO3',right_on='Country Code')
merged2 = merged2.drop(['Country Code_x','Country Code_y'],axis=1)

In [26]:
newColumns=list(merged2.columns)
# Changing names on the merged columns
newColumns[-2] = 'freight_2017'
newColumns[-1] = 'passengers_2017'

# Replacing
merged2.columns = newColumns

The three attributes "SU_A3","GU_A3" and "ADM0_A3" are all unique categories made by Natural Earth, and follows the ISO_A3 mapping, according to [this source](https://www.naturalearthdata.com/forums/topic/thematic-codes/). They allow for a better mapping of the countries, because they go beyond the problem of some countries having an ISO_A3 code of "-99", because they are soverign states. The below aims at justifing the use of one of the unique categories instead of ISO_A3.

In [27]:
len(countries.ISO_A3.unique())

234

In [28]:
len(countries.SU_A3.unique())

241

In [29]:
len(countries.GU_A3.unique())

241

In [30]:
len(countries.ADM0_A3.unique())

241

In [31]:
sum([True if (su == ad) and (gu == ad) and (su == gu) else False\
     for su,ad,gu in zip(countries.SU_A3, countries.GU_A3,countries.ADM0_A3)])

239

In [32]:
for su,ad,gu,iso in zip(countries.SU_A3, countries.GU_A3,countries.ADM0_A3,countries.ISO_A3):
    if (su == ad) and (gu == ad) and (su == gu):
        None
    else:
        print(su,gu,ad,iso)

PR1 PRT PRT PRT
PN1 PNG PNG PNG


In [33]:
for iso,ad,gu,su in zip(countries.ISO_A3,countries.ADM0_A3,countries.GU_A3,countries.SU_A3):
    if iso not in [ad,gu,su]:
        print(iso,ad,gu,su)

SSD SDS SDS SDS
-99 SOL SOL SOL
-99 NOR NOR NOR
ESH SAH SAH SAH
-99 KOS KOS KOS
PSE PSX PSX PSX
-99 FRA FRA FRA
ALA ALD ALD ALD
-99 CYN CYN CYN
-99 IOA IOA IOA
-99 ATC ATC ATC
-99 KAS KAS KAS


In [34]:
combinedCountryData = merged2.merge(countries[[True if country in list(merged2.ISO3) else\
                                               False for country in countries.GU_A3]][['GU_A3','NAME','geometry']],
                                   left_on = 'ISO3',right_on='GU_A3')
combinedCountryData = combinedCountryData.drop(['GU_A3'],axis=1)

In [35]:
combinedCountryData.columns

Index(['organic_products_export', 'chemicals_export', 'material_export',
       'clothing_export', 'metals_export', 'technical_export',
       'transportation_export', 'miscellaneous_goods_export',
       'all_commodities_export', 'organic_products_import', 'chemicals_import',
       'material_import', 'clothing_import', 'metals_import',
       'technical_import', 'transportation_import',
       'miscellaneous_goods_import', 'all_commodities_import', 'ISO3',
       'freight_2017', 'passengers_2017', 'NAME', 'geometry'],
      dtype='object')

### Adding normalised data for visualisation

In [36]:
combinedCountryData['Normalised_export_2017'] = np.log(1+combinedCountryData.all_commodities_export)
combinedCountryData['Normalised_import_2017'] = np.log(1+combinedCountryData.all_commodities_import)
combinedCountryData['Normalised_freight_2017'] = np.log(1+combinedCountryData.freight_2017)
combinedCountryData['Normalised_passengers_2017'] = np.log(1+combinedCountryData.passengers_2017)

### Lower casing all column names

In [37]:
combinedCountryData.columns = [column.lower() for column in combinedCountryData.columns]
combinedCountryData['code'] = combinedCountryData.iso3

In [38]:
gpdCombinedData = gpd.GeoDataFrame(combinedCountryData,geometry=combinedCountryData.geometry)

In [39]:
gpdCombinedData

,organic_products_export,chemicals_export,material_export,clothing_export,metals_export,technical_export,transportation_export,miscellaneous_goods_export,all_commodities_export,organic_products_import,...,iso3,freight_2017,passengers_2017,name,geometry,normalised_export_2017,normalised_import_2017,normalised_freight_2017,normalised_passengers_2017,code
0,0.25,0.02,0.50,0.99,0.35,0.11,0.02,0.06,2.30,0.90,...,ALB,0.00,103151.0,Albania,"POLYGON ((19.34238281250001 41.869091796875, 1...",1.193922,1.835776,0.000000,11.543959,ALB
1,0.35,0.78,33.96,0.00,0.01,0.06,0.02,0.01,35.19,9.42,...,DZA,24.80,6241924.0,Algeria,"POLYGON ((8.576562500000023 36.93720703125, 8....",3.588783,3.851211,3.250374,15.646799,DZA
2,0.00,0.00,0.02,0.01,0.01,0.04,0.02,0.02,0.12,0.34,...,AND,0.00,0.0,Andorra,"POLYGON ((1.7060546875 42.50332031249999, 1.67...",0.113329,0.908259,0.000000,0.000000,AND
3,33.02,11.40,154.37,0.46,10.23,7.67,3.59,3.86,224.60,15.33,...,AUS,1982.59,74257326.0,Australia,"(POLYGON ((143.17890625 -11.95449218750001, 14...",5.418764,5.385045,7.592664,18.123047,AUS
4,12.56,17.69,28.44,4.70,22.40,45.03,19.52,9.51,159.85,13.52,...,AUT,391.27,16171640.0,Austria,"POLYGON ((9.527539062500011 47.270751953125, 9...",5.080472,5.120506,5.971950,16.598770,AUT
5,0.63,0.03,0.97,0.13,0.27,0.04,0.01,0.06,2.14,0.72,...,ARM,0.00,0.0,Armenia,(POLYGON ((44.76826171875001 39.70351562499999...,1.144223,1.587192,0.000000,0.000000,ARM
6,45.21,97.62,107.85,19.18,33.00,44.92,51.44,21.39,420.60,40.16,...,BEL,1513.99,13676844.0,Belgium,"POLYGON ((4.226171875000006 51.386474609375, 4...",6.044057,5.999110,7.323164,16.431215,BEL
7,0.55,0.55,1.61,0.71,1.10,0.74,0.19,0.83,6.28,1.80,...,BIH,0.00,0.0,Bosnia and Herz.,"POLYGON ((19.3486328125 44.880908203125, 19.35...",1.985131,2.436241,0.000000,0.000000,BIH
8,79.26,10.63,70.04,1.70,16.31,17.28,19.84,2.53,217.60,11.20,...,BRA,1736.55,96395709.0,Brazil,(POLYGON ((-66.87602539062499 1.22304687499999...,5.387244,5.022234,7.460231,18.383972,BRA
9,0.01,0.20,5.01,0.01,0.03,0.10,0.04,0.15,5.56,0.47,...,BRN,132.61,1172201.0,Brunei,"(POLYGON ((115.1400390625 4.899755859374991, 1...",1.880991,1.406097,4.894925,13.974395,BRN


## Writing files

In [40]:
# Based on this answer: https://gis.stackexchange.com/questions/159681/geopandas-cant-save-geojson
with open('../Final Data/globalData_reCat.geojson', 'w') as f:
    f.write(gpdCombinedData.to_json())

In [41]:
combinedCountryDataToSave = combinedCountryData.drop(['geometry'],axis=1)
combinedCountryDataToSave.to_csv('../Final Data/globalData_reCat.csv')